In [ ]:
from pyspark import SparkConf
from pyspark.streaming import StreamingContext
from pyspark.sql.session import SparkSession

from pyspark.sql import *
from pyspark.ml.classification import RandomForestClassificationModel, LinearSVCModel
from pyspark.sql.types import IntegerType, FloatType

from pyspark.ml import Pipeline 

In [ ]:
try:
  conf = SparkConf()

  conf.setMaster("spark://10.84.128.47:7077")
  conf.setAppName("G03Streaming")

  spark = SparkSession.builder.config(conf=conf).getOrCreate()

  from kafka import KafkaProducer
  producer = KafkaProducer(bootstrap_servers='10.204.131.11:9092')

  df = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "10.204.131.11:9092") \
    .option("subscribe", "g03in") \
    .option("startingOffsets", "earliest") \
    .load()

  df = df.selectExpr("CAST(value AS STRING)")

  # Split the lines into words, with comma as the separator
  #df = df.select(split(df.value, ";").alias("data"))

  # Transform DataFrame to have "user_id" and "product_id" columns
  # df = df.select(df.data.getItem(0).alias("movie_id"), 
  #              df.data.getItem(1).alias("user_id"), 
  #              df.data.getItem(1).alias("rating"),
  #              df.data.getItem(0).alias("unixtime"))

  # Convert "user_id" and "product_id" to appropriate data types
  # df = df.withColumn("movie_id", df["movie_id"].cast("int"))
  # df = df.withColumn("user_id", df["user_id"].cast("int"))
  # df = df.withColumn("rating", df["rating"].cast("int"))
  # df = df.withColumn("unixtime", df["unixtime"].cast("int"))

  # Add a timestamp column to the dataframe
  # df = df.withColumn("timestamp", current_timestamp())

  try:
    rfcModel = RandomForestClassificationModel.load("hdfs://10.84.129.52:9000/aulas/g03/models/RandomFores/")
    svmModel = LinearSVCModel.load("hdfs://10.84.129.52:9000/aulas/g03/models/SVM/")
  except Exception as e:
    print(str(e))

except Exception as e:
  print(str(e))

def process_batch(df, batchid):
    feature_assembler = VectorAssembler(inputCols=["rating"], outputCol="features")
    prepared_data = feature_assembler.transform(df)
    predictions = svmModel.transform(prepared_data)
    for row in predictions.collect():
    #    print(row)
        message =";".join([str(x) for x in row.asDict().values()])
    #    print(message)
        producer.send('g00out', value=message.encode("utf-8"))

# Write stream out to console
query = df \
    .writeStream \
    .foreachBatch(process_batch) \
    .start()

query.awaitTermination(100)
query.stop()
spark.stop()
print("Job Successfuly Ended!")
